<a href="https://colab.research.google.com/github/henriquevedoveli/classificacao-titulos/blob/master/word2vec_classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificando Títulos de Notícias Utilizando Word2Vec
### ***Notebook de Classificação do Modelo***

---


## ***Objetivo***
#### Desenvolver um classificador capaz de realizar automaticamente o processo de categorização de títulos de notícias.

### Download do core em português (rodar apenas uma vez)

In [1]:
# !python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 1.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=272f9c8b707177c98d1ae97dd3715b32ae8954d347ebabc72f54fe879059f23e
  Stored in directory: /tmp/pip-ephem-wheel-cache-6i13mhvs/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


Bibliotecas Necessárias

In [1]:
import spacy
import numpy as np
from gensim.models import KeyedVectors
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

## Importando os Dados

In [2]:
dados_treino = pd.read_csv('/content/drive/My Drive/data_science/classificacao-titulos/dados/treino.csv')
dados_teste = pd.read_csv('/content/drive/My Drive/data_science/classificacao-titulos/dados/teste.csv')

nlp = spacy.load('pt_core_news_sm', disable = ['paser', 'ner', 'tagger', 'textcat'])

w2v_model_cbow = KeyedVectors.load_word2vec_format('/content/drive/My Drive/data_science/classificacao-titulos/model_cbow.txt')
w2v_model_sg = KeyedVectors.load_word2vec_format('/content/drive/My Drive/data_science/classificacao-titulos/model_sg.txt')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Definindo uma função para tokenizar os textos

#### A função recebe como parâmetro um texto ou um conjunto de texto e retorna os tokens válidos.
####  A função transforma os textos em um doc, que é um objeto da biblioteca spacy. Dentro da função também ocorre um loop dentro dos textos que validam os tokens, os tokens válidos são aqueles que não são stop words e nem alfanúmericos, além disso caso os tokens sejam válidos as letras maiúsculas são transformadas em minúsculas.

---


In [3]:
texto_para_teste = 'TEXTO para o teste das FUnções contendo palavras aleatorias 1235'

In [4]:
def tokenizador(texto):
  doc = nlp(texto)
  tokens_validos = []
  
  for token in doc:
    eh_valido = not token.is_stop and token.is_alpha
    if eh_valido:
      tokens_validos.append(token.text.lower())

  return tokens_validos

#### Rodadando a função para o texto de teste

In [5]:
tokens = tokenizador(texto_para_teste)
print(tokens)

['texto', 'o', 'teste', 'funções', 'contendo', 'palavras', 'aleatorias']


## Definindo uma função para ver o vetor resultante 
#### A função recebe como parâmetro os tokens válidos e o modelo a ser utilizado e retorna um vetor resultante de tamanho 300, com as distâncias das palavras.
---

In [6]:
def combinacao_por_soma(palavras, modelo):
  vetor_r = np.zeros((1,300))

  for token in palavras:
    try:
      vetor_r = vetor_r + modelo.get_vector(token)

    except KeyError:
      pass

  return vetor_r

#### Rodando o função para o texto de teste

In [7]:
vetor_texto = combinacao_por_soma(tokens, w2v_model_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[ 2.58870368e+00  3.61491054e-01  2.88589448e-02 -5.97836580e-01
   8.41937289e-02 -5.23671985e-01 -7.74942279e-01 -1.50745581e+00
  -8.81823326e-01 -1.29006306e+00  2.09286910e-01  7.66616765e-01
   1.40307825e-01 -1.14482539e+00  8.52251083e-01 -5.76672181e-02
   4.11331765e-01 -4.79193963e-01 -4.68215356e-01 -1.20841229e+00
  -2.03771736e+00  1.48951281e+00  1.33954011e+00 -5.30718867e-01
  -7.38969356e-01 -4.79582500e-01 -3.92846901e-01 -1.84166358e-01
  -1.54875653e+00 -1.22219776e+00 -9.96005945e-02  1.99837166e+00
   2.06707408e+00 -9.20886017e-01 -2.35741479e+00 -8.78049500e-01
  -3.15576077e-01 -5.37899429e-01  1.24734840e+00 -8.95041764e-01
  -9.67134908e-01  9.33821984e-01 -5.91737241e-01  7.98646607e-01
   1.14401853e+00  3.66217144e-01  1.03551268e+00 -1.05136736e+00
   6.15798363e-01  6.62204638e-01 -1.11365431e+00  5.10009636e-01
   3.19927744e-01 -9.21370678e-01  3.09843123e-01 -6.97124429e-01
  -7.12333627e-01 -7.35972028e-01 -1.66838919e+00 -2.23825708e-01
 

## Definindo uma função que retorna uma matriz com os vetores

#### A função recebe como parâmetros os textos e o modelo a ser utilizado para o treinamento. A matriz terá o tamanho do tamanho do texto por 300, sendo os elementos as distâncias das palavras.

In [8]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):  
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_por_soma(palavras, modelo)

  return matriz 

#### Construindo a matriz para o modelo CBOW

In [9]:
matriz_vetores_treino_cbow = matriz_vetores(dados_treino['title'], w2v_model_cbow)
matriz_vetores_teste_cbow = matriz_vetores(dados_teste['title'], w2v_model_cbow)

print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


#### Construindo a matriz do modelo Skip-Gram

In [10]:
matriz_vetores_treino_sg = matriz_vetores(dados_treino['title'], w2v_model_sg)
matriz_vetores_teste_sg = matriz_vetores(dados_teste['title'], w2v_model_sg)

print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape)

(90000, 300)
(20513, 300)


## Criando Modelo de Regressão Logística 

#### A função a seguir realiza o treino do Modelo de Regressão Logística recebendo como parâmetro o modelo (CBOW ou Skip-Gram) os dados de treino e os dados de teste.

In [12]:
def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  reg_log = LogisticRegression(max_iter = 800)
  reg_log.fit(x_treino, y_treino)
  categorias = reg_log.predict(x_teste)
  results = classification_report(y_teste, categorias)

  print(results)

  return reg_log

In [13]:
reg_log_cbow = classificador(w2v_model_cbow, 
                        matriz_vetores_treino_cbow, dados_treino['category'],
                        matriz_vetores_teste_cbow, dados_teste['category'])

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.12      0.82      0.22       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.82      0.79      0.80     20513



In [14]:
reg_log_sg = classificador(w2v_model_sg, 
                        matriz_vetores_treino_sg, dados_treino['category'],
                        matriz_vetores_teste_sg, dados_teste['category'])

              precision    recall  f1-score   support

     colunas       0.81      0.72      0.76      6103
   cotidiano       0.65      0.80      0.72      1698
     esporte       0.93      0.88      0.91      4663
   ilustrada       0.14      0.86      0.24       131
     mercado       0.84      0.79      0.81      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.79      0.80     20513



## Salvando os Modelos

In [15]:
with open('/content/drive/My Drive/data_science/classificacao-titulos/rl_cbow.sav','wb') as f:
  pickle.dump(reg_log_cbow, f)

In [16]:
with open('/content/drive/My Drive/data_science/classificacao-titulos/rl_sg.sav','wb') as f:
  pickle.dump(reg_log_sg, f)